## Profile (perfil)

Ya con foto de perfil, vamos a modificar el **`template`** **`profile.html`**, vamos a agregar:

```html
{% extends "blog/base.html" %}
{% load crispy_forms_tags %}
{% block contenido %}
    <div class="content-section">
      <div class="media">
        <img class="rounded-circle account-img" src="{{ user.profile.image.url }}">
        <div class="media-body">
          <h2 class="account-heading">{{ user.username }}</h2>
          <p class="text-secondary">{{ user.email }}</p>
        </div>
      </div>
      <!-- FORM HERE -->
    </div>
{% endblock contenido %}
```

Para que este código funcione, debemos añadir las rutas de las imaganes a la lista **`urlpatterns`** dentro de **`urls.py`** del proyecto.

Hay varias formas de hacerlo, **`django`** tiene documentación sobre esto:

_**href**: https://docs.djangoproject.com/en/4.0/howto/static-files/#serving-files-uploaded-by-a-user-during-development_

Para este ejemplo vamos a modificar **`urls.py`** para que quede de la siguiente manera:
```python
from django.contrib import admin
from django.contrib.auth import views as auth_views
from django.urls import path, include
from users import views as user_views
from django.conf import settings            # Nueva linea
from django.conf.urls.static import static  # Nueva linea

urlpatterns = [
    path('admin/', admin.site.urls),
    path('register/', user_views.register, name = 'register'),
    path('profile/', user_views.profile, name = 'profile'),
    path('login/', auth_views.LoginView.as_view(template_name = 'users/login.html'), name = 'login'),
    path('logout/', auth_views.LogoutView.as_view(template_name = 'users/logout.html'), name = 'logout'),
    path('', include('blog.urls')),
]

# Nuevas lineas
if settings.DEBUG:
    urlpatterns += static(settings.MEDIA_URL, document_root = settings.MEDIA_ROOT)

```

Aquí hemos importado dos nuevos objetos, **`settings`** y **`static`** y los usamos en la condición.

Si nuestro proyecto está en modo de depuración o **`DEBUG`**, se va agregar un nuevo elemento a la lista, un objeto **`static`** que será la ruta para los archivos de imagenes que tengamos de forma local.

Para finalizar esta parte, vamos a agregar la imagen por defecto al directorio **`media`**.

**`Ejecutamos el servidor para ver los perfiles de los usuarios.`**


Ahora vamos a hacer que cada vez que se crea un usuario nuevo, se inicialice un perfil para ese usuario con la imagen por defecto, para esto **`django`** tiene algo llamado **`signals`**.

Vamos al directorio de **`users`** y creamos un nuevo archivo y lo llamamos **`signals.py`** y en el escribimos:

```python
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.contrib.auth.models import User
from .models import Profile

@receiver(post_save, sender=User)
def create_profile(sender, instance, created, **kwargs):
    if created:
        Profile.objects.create(user=instance)
```

**`post_save`** es una señal (**`signal`**) que se dispara cuando un objeto se "guarda", entonces lo que queremos hacer es que se active esta señal cuando un usuario se crea.

**`receiver`** es un decorador que "recibe" la señal y es la encargada de ejecutar lo que queramos hacer. Para usarlo tenemos que indicarle cual es la señal (**`post_save`**) y cual es el objeto que va a iniciar el proceso (**`User`**).

Luego la función que definimos (**`create_profile`**) recibe **`sender`** (**`post_save`**), **`instance`** que sería una instancia de la clase **`User`** y **`created`**, un booleano.


También vamos a crear otra señal que guarde el perfile, la sintaxis es identica:
```python
@receiver(post_save, sender=User)
def save_profile(sender, instance, **kwargs):
    instance.profile.save()
```

Y para que las señales puedan ejecutarse, debemos importar las señales en un método **`ready()`** dentro de **`apps.py`**:
```python
from django.apps import AppConfig

class UsersConfig(AppConfig):
    name = 'users'
    
    def ready(self):
        import users.signals
```

**Para ver si todo esto ha funcionado, ejecutamos el servidor y creamos un usuario y verificamos que tiene un perfil.**